  ## 1. Introduction

* Project

We aim to develop Python-based data analysis software The goal is to develop software that can analyze Micro Led data in detail.

## 2. Project information
 
    
* **Detailed project**
    
  At first,

  The main task is to receive data from the customer and then receive the request.
 
  Main request 
  + Wafer
  + Die row & colum
  + option
 
  We can analyze the selected data by specifying specific elements In order to develop such analysis software, a black box(software) that implments this function is created to solve the customer's request.

  In the software,
  * We process the raw data and print it out. 
  * We show the processed data as a figure and save the file. 
 
 **You can selectively output only the graph you want.**
 
  Please put the data in the 'dat' folder
  specify the file path
  Run the run.py 

* **Run file description** 

   Data is processed by selecting only files in "dat"
   
   Load the module that processes and fits raw data and the module that makes the data parsed from accel, respectively.

## 3. Install and Run 

 * Getting Stared
    + Entered the Terminal, write down 'pip install -r requirements.txt' and download it. \
 ``pip install -r requirements.txt``


 * How to Run
   + Choose the raw data folder customer want to analyze.


## 4. Description of the module file feature


 ### 1) Fitting module 
 
   * The graph is drawn by parsing the raw data of PL data, EL data in the Accel file.
   * The fitting of parsing a raw data and displays the data value y-axis corresponding to x-axis and stored in the graph to visualize the image.

## 5. Example of analysis result

In [2]:
import os

os.chdir('../')
print(os.getcwd())

C:\Users\ASDL\Desktop\micro_led_analysis


  ❗caution❗
>
> If an error occurs, please refresh the jupyter notebook or restart the kernel 

In [3]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import font_manager, rc
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

In [4]:
df = pd.read_excel('./dat/Ver6.xlsx')
df.drop(['No',' X Index',' Y Index',' X Size',' Y Size'],axis=1,inplace=True)

df['PL 장비 JudgeType'].unique()  #array(['PL OK', 'PL NG'], dtype=object)

array(['PL OK', 'PL NG'], dtype=object)

In [5]:
print(df)

     PL 장비 JudgeType     Sum  PL_Average  PL_Max' Value   PL_Min' Value   
0              PL OK  134826          26            134               0  \
1              PL OK  136747          27            130               0   
2              PL OK  138202          27            132               0   
3              PL OK  138073          27            136               0   
4              PL OK  132984          26            130               0   
...              ...     ...         ...            ...             ...   
7585           PL OK  151170          30            137               0   
7586           PL OK  151119          30            137               0   
7587           PL OK  150121          30            137               0   
7588           PL OK  151991          30            137               0   
7589           PL OK  157354          31            142               0   

           EL_PW        EL_PI       EL_DW        EL_II    EL_FWHM   
0     534.015573  2857.285717 

In [6]:
for i in range(0,7590):
    if df.loc[i,'PL 장비 JudgeType'] == 'PL OK':
        df.loc[i,'PL 장비 JudgeType'] = 1
    else:
        df.loc[i,'PL 장비 JudgeType'] = 0

for i in range(0,7590):
    if df.loc[i,'자체 Judge'] == 'EL OK':
        df.loc[i,'자체 Judge'] = 1
        df.loc[i,'Error flag'] = 'No error'
    else:
        df.loc[i,'Error flag'] = df.loc[i,'자체 Judge']
        df.loc[i,'자체 Judge'] = 0

df_1 = pd.read_excel('./dat/Ver6.xlsx')
df_1.drop(['No',' X Index',' Y Index',' X Size',' Y Size'],axis=1,inplace=True)

df_1['PL 장비 JudgeType'].unique()  #array(['PL OK', 'PL NG'], dtype=object)


for i in range(0,7590):
    if df_1.loc[i,'PL 장비 JudgeType'] == 'PL OK':
        df_1.loc[i,'PL 장비 JudgeType'] = 1
    else:
        df_1.loc[i,'PL 장비 JudgeType'] = 0

for i in range(0,7590):
    if df_1.loc[i,'자체 Judge'] == 'EL OK':
        df_1.loc[i,'자체 Judge'] = 1
    else:
        df_1.loc[i,'자체 Judge'] = 0

In [ ]:
# Column distribution by target --> column 별로 데이터 분포를 시각화 해보는 것
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name) 

for cnt, col in enumerate(df):
    try:
        plt.figure(figsize=(10, 5))
        sns.histplot(df[col][df['PL 장비 JudgeType']==1], kde=True, stat="density")
        sns.histplot(df[col][df['PL 장비 JudgeType']==0], kde=True, stat="density")
        plt.legend(['True','False'], loc='best')
        plt.title('histogram of features '+str(col))
        plt.show()

        if cnt >= 15: # 15개 칼럼까지만 출력
            break

    except Exception as e:
        pass

In [ ]:
# 2 Dimension Plot --> 차원축소 기법을 이용하여 2차원으로 데이터를 시각화
X = df.drop(['PL 장비 JudgeType','Error flag'], axis=1)
y = df['PL 장비 JudgeType']

scaler = StandardScaler()
cancer_scale = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)

# plot 2D
from sklearn.decomposition import PCA

pca2 = PCA(n_components=2)
data_pca2 = pca2.fit_transform(cancer_scale)

plt.figure(figsize=(12, 8))
plt.scatter(data_pca2[:,0], data_pca2[:,1], c=df['PL 장비 JudgeType'], s=40, edgecolors='white')
plt.title("2D of Target distribution by PL JudgeType")
plt.xlabel('pcomp 1')
plt.ylabel('pcomp 2')
plt.show()

In [ ]:
# 3 Dimension Plot --> 차원축소 기법을 이용하여 3차원으로 데이터를 시각화
from mpl_toolkits.mplot3d import Axes3D

pca3 = PCA(n_components=3)
data_pca3 = pca3.fit_transform(cancer_scale)

fig = plt.figure(figsize=(12, 8))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(data_pca3[:,0], data_pca3[:,1], data_pca3[:,2], c=df['PL 장비 JudgeType'], s=60, edgecolors='white')
ax.set_title('3D of Target distribution by PL JudgeType')
ax.set_xlabel('pcomp 1')
ax.set_ylabel('pcomp 2')
ax.set_zlabel('pcomp 3')
plt.show()

In [ ]:
# Corrleation Heatmap --> 두 변수가 상관관계가 어떻게 되는지 시각화
font_name = font_manager.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
rc('font', family=font_name) # 한글 출력 설정 부분

cancer_tmp = df_1.copy()
cancer_tmp['PL 장비 JudgeType'] = df_1['PL 장비 JudgeType'].replace({'1':1, '0':0})
corrmat = cancer_tmp.corr()
top_corr_features = corrmat.index[abs(corrmat["PL 장비 JudgeType"])>=0.0]


# plot
plt.figure(figsize=(13,10))
g = sns.heatmap(df[top_corr_features].corr(), annot=True, cmap="RdYlGn")
plt.show()

## 8. Conclusion

 * After passing the law data provided by the customer, they use the ‘fitting module’ to fit.
 
 
 * Save this fitting figure in the ‘res’ folder.
  
  
 - Save the csv file to the 'res' folder based on the data provided by the customer.
 
 
 - After classifing wavelength by wafer and schematically representing the value of Max transmission ref.spec with wild grapes, IV R square was classified into wafers and error flags. Then, the I value at 1V and the I value at -1V are plotted.
 
 
 - And find the standard and variance standard deviations.
 
 
 - In Max transmission ref.spec, the group with the smallest standard deviation is 1310nm of D08, which has a value of 0.39185751314748174.
 

- At 1V, the group with the smallest standard deviation in Current has a value of 0.0001958601941963012 at 1310 nm of D08.


- At -1 V, the group with the smallest standard deviation at current is 1550 nm of D08, which has a value of 9.30988579955451e-10.

 
- Next, the dataframe is briefly represented and the dataframe for the error flag is represented again.

 
- Error Flag fits I-V data. Set the r-sq value to 0. It is expressed as 1, and if it is lower than 0.95, it is expressed as 0.95.

**Therefore of the 98 data, We found 14 files in D23 and 14 files in D24, so please check these data again.**
 